# Bluetooth Low Energy

## Requirements
- adb
- pyshark / tshark

## 1. Data acquisition
### Android HCI logs
Retrieve HCI logs from an Android device with `adb`:

In [ ]:
import subprocess
import datetime

# possible locations and names of the HCI logs file
btsnoop_hci_log_files = [
    {
        'path': '/sdcard/',
        'file_name': 'btsnoop_hci.log'
    },
    {
        'path': '/sdcard/Android/data/',
        'file_name': 'btsnoop_hci.log'
    }
]

btsnoop_hci_dst_folder = 'hci_logs/'
btsnoop_hci_dst_last_file_name = 'hci_log_last.pcap'
btsnoop_hci_dst_file_name = 'hci_log.pcap'

for btsnoop_hci_log_file in btsnoop_hci_log_files:
    try:
        cmd = 'adb pull ' + btsnoop_hci_log_file['path'] + btsnoop_hci_log_file['file_name']  +  ' ' + btsnoop_hci_dst_folder
        subprocess.check_output(cmd, stderr=subprocess.STDOUT, shell=True)
        print('Found: ' + btsnoop_hci_path)
        cmd = 'mv ' + btsnoop_hci_dst_folder + btsnoop_hci_log_file["file_name"] + ' ' + btsnoop_hci_dst_folder + btsnoop_hci_dst_file_name
        subprocess.check_output(cmd, stderr=subprocess.STDOUT, shell=True)
        print('Moved to: ' + btsnoop_hci_dst_folder + btsnoop_hci_dst_last_file_name)
        d = datetime.datetime.today().strftime('%Y%m%d_%H%M')
        cmd = 'cp ' + btsnoop_hci_dst_folder + btsnoop_hci_dst_last_file_name + ' ' + btsnoop_hci_dst_folder + d + '_' + btsnoop_hci_dst_file_name
        subprocess.check_output(cmd, stderr=subprocess.STDOUT, shell=True)
        print('Backed up as: ' + btsnoop_hci_dst_folder + d + '_' + btsnoop_hci_dst_file_name)
        break
    except subprocess.CalledProcessError:
        print('Not found: ' + btsnoop_hci_log_file['path'] + btsnoop_hci_log_file['file_name'])
        pass

### Live capture with Sniffle
Identify the target  with `hcitool lescan`

In [ ]:
try:
    cmd = 'sudo hcitool lescan'
    subprocess.check_output(cmd, stderr=subprocess.STDOUT, shell=True)
    except subprocess.CalledProcessError:

### Load an existing capture


## 2. Data pre-processing

Parse all BLE packets and store only the following BLE ATT packets:
- Read Response
- Write Request
- Write Command
- Value Notification
- Value Indication

In [ ]:
#!/usr/bin/env python3

import pyshark
import binascii

file_name = 'hci_logs/hci_log_last.pcap'

opcodes = {
    0x0b: 'Read Response',
    0x12: 'Write Request',
    0x52: 'Write Command',
    0x1b: 'Value Notification',
    0x1d: 'Value Indication'
}

att_packets=[]

def parse_packet(packet):
    if 'BTATT' in str(packet.layers):
        try:
            att_packet = {}
            att_packet['opcode'] = opcodes.get(int(packet['BTATT'].opcode, 16), 'Unknown')
            att_packet['handle'] = int(packet['BTATT'].handle, 16)
            att_packet['data'] = str(packet['BTATT'].value).replace(':', '')
            #att_packet['data'] = binascii.hexlify(str(packet['BTATT'].value).replace(':', ''))
            #att_packet['data_len'] = len(att_packet['data'])
            att_packets.append(att_packet)
        except AttributeError:
            #print('AttributeError')
            return
        
try:
    capture = pyshark.FileCapture(file_name)
    await capture.packets_from_tshark(parse_packet)

except FileNotFoundError:
    print('File not found: ' + file_name)

if len(att_packets) == 0:
    print('No ATT packets found')
else:
    print('Found: %d ATT packets' % len(att_packets))
    %store att_packets

In [ ]:
#!/usr/bin/env python3

from scapy.all import *
import binascii

file_name = 'hci_logs/20200220_1354_hci_log.pcap'

att_packets = []

def parse_packet(packet):
    packet = BTLE(packet.load)
    
    if packet.haslayer(ATT_Hdr):
        att_packet = {}
        if packet.haslayer(ATT_Write_Request): 
            att_packet['opcode'] = 'Write Request'
            att_packet['gatt_handle'] = packet.getlayer(ATT_Write_Request).gatt_handle
            att_packet['data'] = binascii.hexlify(packet.getlayer(ATT_Write_Request).data)
            att_packets.append(att_packet)
        elif packet.haslayer(ATT_Write_Command): 
            att_packet['opcode'] = 'Write Command'
            att_packet['gatt_handle'] = packet.getlayer(ATT_Write_Command).gatt_handle
            att_packet['data'] = binascii.hexlify(packet.getlayer(ATT_Write_Command).data)
            att_packets.append(att_packet)
        elif packet.haslayer(ATT_Handle_Value_Indication): 
            att_packet['opcode'] = 'Value Indication'
            att_packet['gatt_handle'] = packet.getlayer(ATT_Handle_Value_Indication).gatt_handle
            att_packet['data'] = binascii.hexlify(packet.getlayer(ATT_Handle_Value_Indication).value)
            att_packets.append(att_packet)
        elif packet.haslayer(ATT_Handle_Value_Notification): 
            att_packet['opcode'] = 'Value Notification'
            att_packet['gatt_handle'] = packet.getlayer(ATT_Handle_Value_Notification).gatt_handle
            att_packet['data'] = binascii.hexlify(packet.getlayer(ATT_Handle_Value_Notification).value)
            att_packets.append(att_packet)
        elif packet.haslayer(ATT_Read_Request): 
            print(packet.show())
        

with PcapReader(file_name) as pcap_reader:
    for packet in pcap_reader:
        parse_packet(packet)
        #parse_packet(BTLE(packet.load))

%store att_packets

## 3. Data analysis

In [ ]:
import pandas as pd

df = pd.DataFrame(att_packets)
df = df[['opcode', 'handle', 'data']]
df.style.set_properties(**{'text-align': 'left'})


### Display all packets
Display data sent from the device in `white`, to the device in `black`:

In [ ]:
def highlight_by_handle(row):
    if row.opcode == 'Write Request' or row.opcode == 'Write Command':
        return ['text-align: left; background-color: black; color: white']*3
    else:
        return ['text-align: left']*3

df.style.apply(highlight_by_handle, axis=1)

### Sort packets by handle

In [ ]:
df = df.sort_values(by=['handle'])

# get unique handles
handles = df.handle.unique()
    
# sort handles 
for handle in handles:
    sub_df = df.loc[df['handle'] == handle]
    sub_df = sub_df.sort_index() 
    sub_df = sub_df.style.apply(highlight_by_handle, axis=1)
    #sub_df = sub_df.style.set_properties(**{'text-align': 'left'})
    display(sub_df)

### Sort packets by opcode

In [ ]:
df = df.sort_values(by=['opcode'])

# get unique opcode
opcodes = df.opcode.unique()
    
# sort opcodes 
for opcode in opcodes:
    sub_df = df.loc[df['opcode'] == opcode]
    sub_df = sub_df.sort_index()
    sub_df = sub_df.style.apply(highlight_by_handle, axis=1)
    #sub_df = sub_df.style.set_properties(**{'text-align': 'left'})
    display(sub_df)